In [2]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import sys
sys.path.append('gwas-trans-eqtl-analysis/analysis/utils/')
import myutils
import pandas as pd
import numpy as np
import os

loading_file = "datasets/hore_2016/gene_loadings.txt"
df = pd.read_csv(loading_file, header=0, index_col=0, sep="\t")

In [3]:
dbsnp_catalog_path = "datasets/gtex_v8/metadata/variant_b37_rs_id_dbSNP151_GRCh38p7_map.tsv"
# maps variant id to rsid sbSNP151 GRCh38p7
dbsnp_dict, variantid_dict = myutils.get_id_dictionaries(dbsnp_catalog_path)

In [4]:
import gzip
def load_snpinfo(snpinfofile):
    snp_list = list()
    with gzip.open(snpinfofile) as inpt:
        for line in inpt:
            arr = line.decode().strip().split(" ")
            var_id = arr[1]
            snp_list.append(var_id)
    return snp_list

snpinfofile = "datasets/gtex_v8/gtex_v8_snpinfo_SHAPEIT2.txt.gz"
snps_list = load_snpinfo(snpinfofile)
rsid_list = [dbsnp_dict[x] for x in snps_list]
tejaas_rsids = dict(zip(rsid_list, snps_list))

In [6]:
paper_rsids = ["rs194862","rs9924520","rs289749","rs6882516","rs2401506","rs17611866","rs12630796"]
df[paper_rsids]


KeyError: "['rs17611866', 'rs9924520', 'rs194862', 'rs6882516', 'rs12630796'] not in index"

In [8]:
paper_rsids = ["rs194862","rs9924520","rs289749","rs6882516","rs2401506","rs17611866","rs12630796"]
#in ld rs13320918 rs2553419 rs2682386 rs9859077 rs2141180

# for rsid in paper_rsids:
counter = 0
counter_paper = 0
for rsid in list(df.columns)+paper_rsids:
    if rsid.startswith("rs"):
        if rsid in paper_rsids:
            print(rsid, variantid_dict[rsid])
        if rsid in tejaas_rsids:
            counter += 1
            if rsid in paper_rsids:
                counter_paper += 1
                print("in paper ", rsid)
            
print(f"analyzed {counter} snps of the {df.shape[1]} SNPs in Hore")
print(f"{counter_paper} from main Hore paper")

rs2401506 chr22_18144963_C_G_b38
rs289749 chr16_56992382_G_A_b38
in paper  rs289749
rs194862 chr7_104145832_T_C_b38
in paper  rs194862
rs9924520 chr16_10880498_G_A_b38
in paper  rs9924520
rs289749 chr16_56992382_G_A_b38
in paper  rs289749
rs6882516 chr5_157755889_C_A_b38
in paper  rs6882516
rs2401506 chr22_18144963_C_G_b38
rs17611866 chr16_3317229_T_C_b38
in paper  rs17611866
rs12630796 chr3_101512090_C_G_b38
analyzed 715 snps of the 944 SNPs in Hore
6 from main Hore paper


In [10]:
### 1st, check how many snps from Hore we replicate (in the same 3 tissues)
### If we replicate some, then
### 2nd, run Tejaas with beta values only for them
import collections

basepath = "/home/franco/soedinglab"
basepath_data = os.path.join(basepath, "datasets")
basepath_results = os.path.join("gwas-trans-eqtl-analysis", "results")
basepath_gtex = os.path.join(basepath_data, "gtex_v8")

gamma1_file = "protein_coding_lncRNA_gamma01_knn30_cut5e-8"
optim_file = "protein_coding_lncRNA_optim_gamma_knn30_cut5e-8"

gamma1 = "gamma01_knn30_cut5e-8"
optim = "optim_gamma_knn30_cut5e-8"
basepath_gamma1_trans_eqtls = os.path.join(basepath_gtex, gamma1_file)
basepath_optim_trans_eqtls = os.path.join(basepath_gtex, optim_file)

chromlist = range(1,23)

tissue_file    = os.path.join("gwas-trans-eqtl-analysis/analysis/tissue_table.txt")
gtex_meta_file = os.path.join("../../../../trans-eqtl-manuscript/figures/results/gtex/gtex_v8_metadata.json")
optim_tissues = ['ag', 'haa', 'liv', 'msg', 'pan', 'pit', 'si', 'spl', 'va', 'wb']

tshorts, tfulls, tstrings = myutils.read_tissues(tissue_file)
tissues_string = dict(zip(tshorts, tstrings))
tissues_full = dict(zip(tshorts, tfulls))
tissues_abbr = dict(zip(tfulls, tshorts))

tissue_eqtls_gamma1, tissue_eqtls_ld_gamma1 = myutils.read_trans_eqtls(basepath_gamma1_trans_eqtls, 
                                                                        tshorts, 
                                                                        dbsnp_dict)
tissue_eqtls_optim, tissue_eqtls_ld_optim = myutils.read_trans_eqtls(basepath_optim_trans_eqtls, 
                                                                        tshorts, 
                                                                        dbsnp_dict)

tissue_eqtls_ld = collections.defaultdict(list)
tissue_eqtls_all = collections.defaultdict(list)
for tissue in tshorts:
    if tissue in optim_tissues:
        tissue_eqtls_ld[tissue] = tissue_eqtls_ld_optim[tissue]
        tissue_eqtls_all[tissue] = tissue_eqtls_optim[tissue]
    else:
        tissue_eqtls_ld[tissue] = tissue_eqtls_ld_gamma1[tissue]
        tissue_eqtls_all[tissue] = tissue_eqtls_gamma1[tissue]
        
rsids_ld = collections.defaultdict(set)
for tissue in tshorts:
    for entry in tissue_eqtls_ld[tissue]:
        rsids_ld[tissue].add(entry.dbsnp)
        
rsids_all = collections.defaultdict(set)
for tissue in tshorts:
    for entry in tissue_eqtls_all[tissue]:
        rsids_all[tissue].add(entry.dbsnp)

as has 2011 trans-eqtls with 586 lead variants
av has 1664 trans-eqtls with 463 lead variants
ag has 822 trans-eqtls with 184 lead variants
aa has 5760 trans-eqtls with 1298 lead variants
ac has 7551 trans-eqtls with 1539 lead variants
at has 1463 trans-eqtls with 391 lead variants
bam has 4780 trans-eqtls with 1182 lead variants
ban has 3890 trans-eqtls with 954 lead variants
bca has 27 trans-eqtls with 19 lead variants
bceh has 270 trans-eqtls with 128 lead variants
bce has 96 trans-eqtls with 40 lead variants
bco has 113 trans-eqtls with 64 lead variants
bfr has 58 trans-eqtls with 32 lead variants
bhi has 170 trans-eqtls with 51 lead variants
bhy has 325 trans-eqtls with 83 lead variants
bnu has 1451 trans-eqtls with 379 lead variants
bpu has 14 trans-eqtls with 10 lead variants
bsp has 186 trans-eqtls with 61 lead variants
bsu has 202 trans-eqtls with 28 lead variants
br has 1764 trans-eqtls with 505 lead variants
ebv has 1241 trans-eqtls with 297 lead variants
fib has 767 trans-e

In [11]:
hore_tissues = ["ebv", "as", "sse", "snse"]

for tissue in tshorts:
    myrsids = list(rsids_all[tissue])
    for rsid in df.columns:
        # for rsid in paper_rsids:
        if rsid.startswith("rs"):
            if rsid in myrsids:
                print(f"{tissue} {rsid} replicated!")
                if rsid in paper_rsids:
                    print("in paper!!")

av rs1932201 replicated!
ag rs3817362 replicated!
ag rs1932201 replicated!
ag rs1792218 replicated!
ag rs2405143 replicated!
ag rs4742478 replicated!
ag rs11641185 replicated!
aa rs11641185 replicated!
liv rs3817362 replicated!
liv rs740000 replicated!
ms rs3817362 replicated!
ms rs11641185 replicated!
ms rs1851555 replicated!
ms rs740000 replicated!
si rs3817362 replicated!
si rs1932201 replicated!
si rs2217285 replicated!
si rs2405143 replicated!
si rs10050872 replicated!
si rs1028695 replicated!
si rs4742478 replicated!
spl rs3817362 replicated!
spl rs12531170 replicated!
spl rs1474404 replicated!
tes rs3817362 replicated!
tes rs740000 replicated!
ut rs2134180 replicated!
wb rs12941904 replicated!
wb rs9915548 replicated!
wb rs11641185 replicated!


In [37]:
import readgtf

gencode_file = os.path.join(basepath_data, "gencode.v26.annotation.gtf.gz")
geneinfo = readgtf.gencode_v12(gencode_file)

ensembl_gene_symbol_map = dict()
for gene in geneinfo:
    ensembl_id = gene.ensembl_id.split(".")[0]
    ensembl_gene_symbol_map[ensembl_id] = gene.name

In [42]:
# Trans-eQTLs Variant ids for each tissue
# For each trans-eQTL the ensemblIDs of target genes as list
target_genes = collections.defaultdict(lambda: collections.defaultdict(list))
for tissue in tshorts:
    if tissue in optim_tissues:
        basepath_trans_eqtls = basepath_optim_trans_eqtls
    else:
        basepath_trans_eqtls = basepath_gamma1_trans_eqtls
    
    # Getting the tejaas file belongig to the tissue with the calculated target genes for all the trans-eQTLs found
    target_gene_file = os.path.join(basepath_trans_eqtls, tissue, "target_genes.txt")
    if os.path.exists(target_gene_file):
        with open(target_gene_file) as fin:
            next(fin)
            for line in fin:
                arr = line.split("\t")
                variant_id = arr[0]
                # getting rid of version number eg. ENSG00000009780.15 becomes ENSG00000009780
                ensembl_id = arr[1].split(".")[0]
                target_genes[tissue][variant_id].append(ensembl_id)

In [44]:
target_genes['wb'][variantid_dict['rs11641185']]

['ENSG00000117616',
 'ENSG00000076003',
 'ENSG00000188282',
 'ENSG00000206561',
 'ENSG00000248019',
 'ENSG00000138640',
 'ENSG00000181381',
 'ENSG00000106483',
 'ENSG00000189233',
 'ENSG00000108091',
 'ENSG00000119705',
 'ENSG00000100603',
 'ENSG00000270240',
 'ENSG00000271503',
 'ENSG00000161381',
 'ENSG00000274173']